In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\STARS\AppData\Local\Temp\ipykernel_13836\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [4]:
data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


### Let's divide the training and test set into two partitions

In [5]:
from sklearn.model_selection import train_test_split
x=data['text']
y=data['label']
y_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

## Data Preprocessing

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\STARS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [8]:
import re

def clean_html(text):
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL)
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

data["clean_text"] = data["text"].apply(clean_html)

print(data[["text", "clean_text"]].head())


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                          clean_text  
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...  
1                                           Will do.  
2  Nora--Cheryl has emailed dozens of memos about...  
3  Dear Sir=2FMadam=2C I know that this proposal ...  
4                                                fyi  


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [9]:
import re

def preprocess_text(text):
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL)
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    text = re.sub(r'^\s*[a-zA-Z]\s+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\bb\b', '', text)
    return text.lower()

data["clean_text"] = data["text"].apply(preprocess_text)

print(data[["text", "clean_text"]].head())


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                          clean_text  
0  dear sir strictly private business proposal am...  
1                                            will do  
2  noracheryl has emailed dozens of memos about h...  
3  dear sirfmadamc know that this proposal might ...  
4                                                fyi  


## Now let's work on removing stopwords
Remove the stopwords.

In [10]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL)
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    text = re.sub(r'^\s*[a-zA-Z]\s+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\bb\b', '', text)
    text = text.lower()
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

data["clean_text"] = data["text"].apply(preprocess_text)

print(data[["text", "clean_text"]].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\STARS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                          clean_text  
0  dear sir strictly private business proposal mi...  
1                                                     
2  noracheryl emailed dozens memos haiti weekend ...  
3  dear sirfmadamc know proposal might surprise e...  
4                                                fyi  


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [11]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL)
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    text = re.sub(r'^\s*[a-zA-Z]\s+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\bb\b', '', text)
    text = text.lower()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

data["clean_text"] = data["text"].apply(preprocess_text)

print(data[["text", "clean_text"]].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\STARS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\STARS\AppData\Roaming\nltk_data...


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                          clean_text  
0  dear sir strictly private business proposal mi...  
1                                                     
2  noracheryl emailed dozen memo haiti weekend pl...  
3  dear sirfmadamc know proposal might surprise e...  
4                                                fyi  


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [12]:
from collections import Counter

# Separate ham and spam messages
ham_words = " ".join(data[data["label"] == 0]["clean_text"]).split()
spam_words = " ".join(data[data["label"] == 1]["clean_text"]).split()

# Get the 10 most common words in ham and spam messages
ham_top_words = Counter(ham_words).most_common(10)
spam_top_words = Counter(spam_words).most_common(10)

print("Top 10 words in HAM messages:")
print(ham_top_words)

print("\nTop 10 words in SPAM messages:")
print(spam_top_words)


Top 10 words in HAM messages:
[('u', 115), ('pm', 115), ('would', 106), ('state', 103), ('president', 94), ('call', 91), ('time', 84), ('percent', 77), ('secretary', 76), ('work', 73)]

Top 10 words in SPAM messages:
[('money', 920), ('account', 794), ('bank', 745), ('fund', 703), ('u', 550), ('business', 473), ('transaction', 416), ('country', 406), ('transfer', 392), ('million', 385)]


## Extra features

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)

money_simbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"])

for dataset in [data_train, data_val]:
    dataset['money_mark'] = dataset['clean_text'].str.contains(money_simbol_list, regex=True, na=False).astype(int)
    dataset['suspicious_words'] = dataset['clean_text'].str.contains(suspicious_words, regex=True, na=False).astype(int)
    dataset['text_len'] = dataset['clean_text'].apply(len)

data_train.head()


,text,label,clean_text,money_mark,suspicious_words,text_len
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,regard mr nelson smithkindly reply private ema...,1,0,75
535,I have not been able to reach oscar this am. W...,0,able reach oscar supposed send pdb receive,1,0,42
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,huma abedin bim checking pat work jack jake re...,1,0,79
557,I can have it announced here on Monday - can't...,0,announced monday cant today,1,0,27
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,bank africaagence san pedro bp san pedro cote ...,1,1,1067


## How would work the Bag of Words with Count Vectorizer concept?

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the text data
X_train_bow = vectorizer.fit_transform(data_train["clean_text"])
X_val_bow = vectorizer.transform(data_val["clean_text"])

# Get the feature names (words in vocabulary)
feature_names = vectorizer.get_feature_names_out()

# Convert to DataFrame for better visualization
bow_df = pd.DataFrame(X_train_bow.toarray(), columns=feature_names)

print("Bag of Words Representation (first 5 rows):")
print(bow_df.head())


Bag of Words Representation (first 5 rows):
   aac  aaclocated  aae  aag  aaronovitchon  abacha  abachabefore  abachac  \
0    0           0    0    0              0       0             0        0   
1    0           0    0    0              0       0             0        0   
2    0           0    0    0              0       0             0        0   
3    0           0    0    0              0       0             0        0   
4    0           0    0    0              0       0             0        0   

   abachace  abachaco  ...  zongo  zongothe  zuhair  \
0         0         0  ...      0         0       0   
1         0         0  ...      0         0       0   
2         0         0  ...      0         0       0   
3         0         0  ...      0         0       0   
4         0         0  ...      0         0       0   

   zulatoebikozulatonetscapeenet  zulatoffffffffffffffffff  zuma  zumac  \
0                              0                         0     0      0   
1     

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Vectorize the entire dataset
X_train_tfidf = tfidf_vectorizer.fit_transform(data_train["clean_text"])
X_val_tfidf = tfidf_vectorizer.transform(data_val["clean_text"])

# Print the shape of the vectorized dataset
print("Shape of TF-IDF vectorized training set:", X_train_tfidf.shape)
print("Shape of TF-IDF vectorized validation set:", X_val_tfidf.shape)


Shape of TF-IDF vectorized training set: (800, 16932)
Shape of TF-IDF vectorized validation set: (200, 16932)


## And the Train a Classifier?

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Train the classifier using the TF-IDF vectorized data
clf = LogisticRegression(max_iter=1000)

# Fit the model on the training data
clf.fit(X_train_tfidf, data_train['label'])

# Predict on the validation set
y_pred = clf.predict(X_val_tfidf)

# Evaluate the model
accuracy = accuracy_score(data_val['label'], y_pred)
print("Accuracy on validation set:", accuracy)

# Print detailed classification report
print("Classification Report:")
print(classification_report(data_val['label'], y_pred))


Accuracy on validation set: 0.96
Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       125
           1       1.00      0.89      0.94        75

    accuracy                           0.96       200
   macro avg       0.97      0.95      0.96       200
weighted avg       0.96      0.96      0.96       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code